# Create a Database

In [7]:
# import modules
import psycopg2
from psycopg2 import sql

In [8]:
connection_params = {
    "host": "localhost",
    "user": "postgres",
    "password": "postgres",
    "port": "5432"
}

In [9]:
db_name = 'socialmedia'
sql_file_path = './schema.sql'

In [10]:
conn = psycopg2.connect(**connection_params)
conn.autocommit = True  # Enable autocommit to execute DDL statements

# Create a new database
with conn.cursor() as cursor:
    create_db_query = sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name))
    cursor.execute(create_db_query)

# Close the connection to the default database
conn.close()

Check that the database was created!

In [11]:
# Connect to the newly created database
connection_params["database"] = db_name
conn = psycopg2.connect(**connection_params)
conn.autocommit = True

# Execute the SQL file to define the schema
with conn.cursor() as cursor:
    with open(sql_file_path, "r") as file:
        sql_file_content = file.read()
        cursor.execute(sql_file_content)

# Close the connection to the new database
conn.close()

Check that empty tables were created in the database!

# Add Data

In [12]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

In [ ]:
# connection_params = {
#     "host": "localhost",
#     "user": "postgres", # username
#     "password": "postgres", 
#     "dbname": "demo_db",
#     "port": "5432"
# }

## Create DataFrames

In [13]:
users_data = {
    'username': ['Alice', 'Bob', 'Charlie'],
    'email': ['alice@example.com', 'bob@example.com', 'charlie@example.com'],
    'created_at': [datetime.now(), datetime.now(), datetime.now()]
}

posts_data = {
    'user_id': [1, 2, 1],
    'title': ['Post 1', 'Post 2', 'Another Post'],
    'content': ['Content of Post 1', 'Content of Post 2', 'Lorem ipsum dolor sit amet.'],
    'created_at': [datetime.now(), datetime.now(), datetime.now()]
}

users_df = pd.DataFrame(users_data)
posts_df = pd.DataFrame(posts_data)

In [14]:
users_df

,username,email,created_at
0,Alice,alice@example.com,2023-11-29 16:35:09.975719
1,Bob,bob@example.com,2023-11-29 16:35:09.975724
2,Charlie,charlie@example.com,2023-11-29 16:35:09.975725


In [15]:
posts_df

,user_id,title,content,created_at
0,1,Post 1,Content of Post 1,2023-11-29 16:35:09.975799
1,2,Post 2,Content of Post 2,2023-11-29 16:35:09.975800
2,1,Another Post,Lorem ipsum dolor sit amet.,2023-11-29 16:35:09.975801


## Add data from DataFrames to Database

In [18]:
connection_params

{'host': 'localhost',
 'user': 'postgres',
 'password': 'postgres',
 'port': '5432',
 'database': 'socialmedia'}

In [19]:
engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")

# Upload 'users' DataFrame to PostgreSQL
users_df.to_sql('users', con=engine, if_exists='replace', index=False)

# Upload 'posts' DataFrame to PostgreSQL
posts_df.to_sql('posts', con=engine, if_exists='append', index=False)

3

Now, check that tables again.